In [1]:
!pip install pylidc
!pip install pydicom
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 20.2 MB/s eta 0:00:0000:0100:01


In [2]:
import numpy as np
import pylidc as pl
import pydicom
import cv2
from statistics import median_high
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, BatchNormalization, Activation, AveragePooling2D, concatenate
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint, EarlyStopping

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix, classification_report, recall_score, f1_score


In [3]:
import os
import pydicom
import pylidc as pl
import numpy as np
import cv2
from pylidc.Contour import Contour
from statistics import median_high

# Paths to the directories containing DICOM files
base_dirs = [
    "/kaggle/input/lidcidri30/LIDC-IDRI-0001-0200",
    "/kaggle/input/lidcidri30/LIDC-IDRI-0201-0400",
    "/kaggle/input/lidcidri30/LIDC-IDRI-0401-0600"
]

images = []
labels = []
malignant_count = 0
benign_count = 0
max_malignant_count = 4500  # Target number of malignant images
max_benign_count = 2500     # Target number of benign images

# Patch the Contour class method to_matrix
def patched_to_matrix(self, include_k=True):
    coords = np.array([list(map(int, coord.split(','))) for coord in self.coords.split('\n')])
    ij = np.c_[coords[:, 0], coords[:, 1]]

    if not include_k:
        return ij.astype(np.int64)

    k = np.ones(ij.shape[0]) * self.image_z_position
    return np.c_[ij, k].astype(np.int64)

Contour.to_matrix = patched_to_matrix

def calculate_malignancy(nodule):
    list_of_malignancy = [annotation.malignancy for annotation in nodule]
    malignancy = median_high(list_of_malignancy)
    return 'malignant' if malignancy > 3 else 'benign'

# Process DICOM files with the patched Contour class
for base_dir in base_dirs:
    for dirpath, _, filenames in os.walk(base_dir):
        for filename in filenames:
            if filename.endswith(".dcm"):
                try:
                    dicom_file = os.path.join(dirpath, filename)
                    dicom_data = pydicom.dcmread(dicom_file)
                    pid = dicom_data.PatientID
                    scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == pid).first()

                    if scan is None:
                        continue

                    nodules_annotation = scan.cluster_annotations()

                    try:
                        if len(nodules_annotation) > 0:
                            contains_malignant = False

                            for nodule in nodules_annotation:
                                malignancy = calculate_malignancy(nodule)
                                if malignancy == 'malignant':
                                    contains_malignant = True
                                    break

                            if contains_malignant:
                                if malignant_count >= max_malignant_count:
                                    continue
                                malignancy = 'malignant'
                                malignant_count += 1
                            else:
                                if benign_count >= max_benign_count:
                                    continue
                                malignancy = 'benign'
                                benign_count += 1

                            img = dicom_data.pixel_array  # Assuming DICOM contains pixel data directly
                            img = cv2.resize(img, (224, 224))  
                            img = img.astype(np.float32)

                            images.append(img)
                            labels.append(malignancy)

                            print(f"Processed {malignant_count + benign_count} files")
                            print(f"Malignancy: {malignancy}")
                            print(f"Patient ID: {pid}")

                        if malignant_count >= max_malignant_count and benign_count >= max_benign_count:
                            break
                    except Exception as e:
                        # Skip problematic nodules that cannot be grouped manually
                        print(f"Skipping nodule processing for Patient ID {pid} due to error: {e}")

                except Exception as e:
                    print(f"Error processing file {filename}: {e}")

        if malignant_count >= max_malignant_count and benign_count >= max_benign_count:
            break

print(f"Total malignant images: {malignant_count}")
print(f"Total benign images: {benign_count}")


Processed 1 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 2 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 3 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 4 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 5 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 6 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 7 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 8 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 9 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 10 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 11 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 12 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 13 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 14 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 15 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 16 files
Malignancy: ben

In [4]:
# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

In [5]:
# Normalization
images = images/255.0

In [6]:
# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_encoded = to_categorical(labels_encoded)

# Print the mapping of labels to their encoded values
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Mapping:", label_mapping)


Label Mapping: {'benign': 0, 'malignant': 1}


In [7]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(images, labels_encoded, test_size=0.2, random_state=42, shuffle=True)

# Convert to numpy arrays and reshape
X_train = np.array(X_train).reshape(-1, 224, 224, 1)
X_test = np.array(X_test).reshape(-1, 224, 224, 1)

# Define number of classes
num_classes = len(np.unique(labels_encoded))

print(num_classes)

2


In [8]:
import tensorflow as tf

input_shape = (224, 224, 1)  # Input shape for grayscale images

model = tf.keras.Sequential([
    # Resizing layer to resize the input images
    tf.keras.layers.Resizing(224, 224, interpolation='bilinear', input_shape=input_shape),
    
    # Block 1
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2)),

    # Block 2
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2)),

    # Block 3
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2)),

    # Block 4
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2)),

    # Block 5
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2)),

    # Classification head
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2, activation='softmax')
])


/opt/conda/lib/python3.10/site-packages/keras/src/layers/preprocessing/tf_data_layer.py:18: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [9]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resizing (Resizing)             │ (None, 224, 224, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 224, 224, 64)   │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4096)           │             

 Total params: 139,577,282 (532.45 MB)

 Trainable params: 139,577,282 (532.45 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, load_model

# Assuming ResNet50 and other utility functions (identity_block, convolutional_block) are defined.

# Number of folds
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Placeholder array to store performance metrics
accuracy_scores = []

# Perform cross-validation
for fold, (train_index, val_index) in enumerate(kf.split(X_train)):
    print(f'Fold {fold + 1}/{k}')
    
    # Split data into training and validation sets for the fold
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    # Initialize a new model for each fold
    model = tf.keras.Sequential([
    # Resizing layer to resize the input images
    tf.keras.layers.Resizing(224, 224, interpolation='bilinear', input_shape=input_shape),
    
    # Block 1
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2)),

    # Block 2
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2)),

    # Block 3
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2)),

    # Block 4
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2)),

    # Block 5
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2)),

    # Classification head
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2, activation='softmax')
])

    
   # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

    # Filepath to save the best model during training
    checkpoint_filepath = f'best_vgg_model_fold_{fold + 1}.keras'
    
    # ModelCheckpoint to save the best weights during training
    checkpoint = ModelCheckpoint(checkpoint_filepath, 
                                 monitor='val_accuracy', 
                                 save_best_only=True, 
                                 mode='max', 
                                 verbose=1)

    # EarlyStopping to stop training if validation accuracy does not improve
    early_stopping = EarlyStopping(monitor='val_accuracy', 
                                   patience=4,  # Stop training if no improvement after 4 epochs
                                   mode='max', 
                                   verbose=1, 
                                   restore_best_weights=True)

    # Train the model
    model.fit(X_train_fold, y_train_fold, 
              epochs=5,  # You can increase the number of epochs if needed
              batch_size=16, 
              validation_data=(X_val_fold, y_val_fold),
              verbose=1,
              callbacks=[checkpoint, early_stopping])

    # Load the best model saved during this fold
    model = load_model(checkpoint_filepath)

    # Evaluate the model on the validation set
    scores = model.evaluate(X_val_fold, y_val_fold, verbose=0,batch_size=4)
    accuracy_scores.append(scores[1])  # Save validation accuracy for this fold

# Calculate the average accuracy across all folds
avg_accuracy = np.mean(accuracy_scores)
print(f'Average Accuracy across {k} folds: {avg_accuracy:.4f}')


Fold 1/5
Epoch 1/5


I0000 00:00:1727155495.485425     141 service.cc:145] XLA service 0x7cf5b4023e90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727155495.485494     141 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1727155495.485500     141 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1727155523.168083     141 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


280/280 ━━━━━━━━━━━━━━━━━━━━ 0s 264ms/step - accuracy: 0.6327 - loss: 0.6628
Epoch 1: val_accuracy improved from -inf to 0.65893, saving model to best_vgg_model_fold_1.keras
280/280 ━━━━━━━━━━━━━━━━━━━━ 120s 312ms/step - accuracy: 0.6328 - loss: 0.6627 - val_accuracy: 0.6589 - val_loss: 0.6362
Epoch 2/5
280/280 ━━━━━━━━━━━━━━━━━━━━ 0s 282ms/step - accuracy: 0.6494 - loss: 0.6429
Epoch 2: val_accuracy improved from 0.65893 to 0.71071, saving model to best_vgg_model_fold_1.keras
280/280 ━━━━━━━━━━━━━━━━━━━━ 92s 328ms/step - accuracy: 0.6495 - loss: 0.6428 - val_accuracy: 0.7107 - val_loss: 0.5438
Epoch 3/5
280/280 ━━━━━━━━━━━━━━━━━━━━ 0s 284ms/step - accuracy: 0.7618 - loss: 0.4588
Epoch 3: val_accuracy improved from 0.71071 to 0.96071, saving model to best_vgg_model_fold_1.keras
280/280 ━━━━━━━━━━━━━━━━━━━━ 92s 330ms/step - accuracy: 0.7620 - loss: 0.4584 - val_accuracy: 0.9607 - val_loss: 0.1611
Epoch 4/5
280/280 ━━━━━━━━━━━━━━━━━━━━ 0s 285ms/step - accuracy: 0.9464 - loss: 0.1481
Epoc